In [1]:
import cv2
import numpy as np

In [2]:
def img_resize(img,width=None,height=None,inter=cv2.INTER_AREA):
    dim=None
    (h,w)=img.shape[:2]
    if width is None and height is None:
        return
    if width is None:
        r=height/float(h)
        dim=(int(w*r),height)
    else:
        r=width/float(w)
        dim=(width,int(h*r))
    return cv2.resize(img,dim,interpolation=inter)

In [4]:
first_frame=None

cap=cv2.VideoCapture(0)

while True:
    _,frame=cap.read()
    frame=img_resize(frame,width=500)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(21,21),0)
    
    if first_frame is None:
        first_frame=gray
        continue
    
    diff=cv2.absdiff(gray,first_frame)
    thresh=cv2.threshold(diff,25,255,cv2.THRESH_BINARY)[1]
    thresh=cv2.dilate(thresh,None,iterations=2)
    cnts,hier=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in cnts:
        if cv2.contourArea(c)<10000:
            continue
        (x,y,w,h)=cv2.boundingRect(c)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),6)
        
    cv2.imshow('Difference',diff)
    cv2.imshow('Thresh',thresh)
    cv2.imshow("Out",frame)
    
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()